<a href="https://colab.research.google.com/github/rahmanurannisa/absa/blob/main/TrainSVM8020v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Depedencies

In [ ]:
# !pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 9.3 MB/s eta 0:00:00


In [ ]:
# pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import pickle

import nltk

# import torch

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# download stopwords
nltk.download('stopwords')
nltk.download('punkt')

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# from sklearn.naive_bayes import MultinomialNB

# from transformers import BertTokenizer, BertModel

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
#logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt
# % matplotlib inline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

#BERT
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
# from torch.utils.data import TensorDataset, DataLoader
# from transformers import get_linear_schedule_with_warmup

from sklearn.metrics import accuracy_score, confusion_matrix

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Data Preparation

## 1.1 Load The Dataset

This is the primary step of the entire pipeline. In this case, we have to load the dataset using pandas.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/FoodQualityABSA/DatasetKualitasMakananSensoryValid.csv')

df

,no,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma
0,1,"Sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
1,2,Strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
2,3,Sometimes when you feel super hungry especiall...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE
3,4,Enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
4,5,"Sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...
18612,18613,Nyaman tempatnya dan luas Masakan nya enak Har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE
18613,18614,Kualitas makanam biasa. Suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE
18614,18615,Ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE
18615,18616,Restoran suasana saung sunda dengan makanan ya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,POSITIF,NONE,NONE


## 1.2 Validating The Number of Entries

This includes the number of entries we have in the dataset.

In [ ]:
len(df), df.index.shape[-1]

(18617, 18617)

## 1.3 The Shape of The Dataset

Assess the shape of the data.

In [ ]:
print("The shape of the dataset: " .format(), df.shape)

The shape of the dataset:  (18617, 12)


## 1.4 Statistical Analysis

This is the start of the analysis phase where we will first check the amount of data present in either of the sentiments. We will follow this up with some pictorial representations related to the words and frequency mappings.

### Type of Food

In [ ]:
jumlah_makanan = df['jenis_makanan'].nunique()
print("Jumlah Makanan:")
print(jumlah_makanan)

print(" ")

jenis_makanan = df['jenis_makanan'].unique()
print("Jenis Makanan:")
print(jenis_makanan)

Jumlah Makanan:
34
 
Jenis Makanan:
['Soto' 'Pecel Lele' 'Sate' 'All you can eat' 'Makanan Harian' 'Ayam '
 'Fast Food' 'Masakan Padang' 'Seafood' 'Italian food' 'Chinese food'
 'Bubur' 'Masakan Manado' 'Bakmi' 'Masakan Nusantara' 'France Food'
 'Sundanese Food' 'Sop ' 'Steak' 'Sulawesi Food' 'Makanan Asia'
 'Snack Nusantara' 'Makanan korea' 'Olahan Kambing' 'Bebek' 'Gado-gado'
 'Japanese Food' 'Garang Asem' 'Roti ' 'Ayam & Bebek' 'Dimsum' 'Sushi'
 'Nasi Pecel' 'Martabak']


In [ ]:
x = 0
for makanan in jenis_makanan:
    print(f"Jumlah ulasan makanan {jenis_makanan[x]}: {df['jenis_makanan'].value_counts().loc[jenis_makanan[x]]}")
    x += 1

Jumlah ulasan makanan Soto: 1718
Jumlah ulasan makanan Pecel Lele: 2
Jumlah ulasan makanan Sate: 466
Jumlah ulasan makanan All you can eat: 402
Jumlah ulasan makanan Makanan Harian: 941
Jumlah ulasan makanan Ayam : 695
Jumlah ulasan makanan Fast Food: 767
Jumlah ulasan makanan Masakan Padang: 2160
Jumlah ulasan makanan Seafood: 1022
Jumlah ulasan makanan Italian food: 255
Jumlah ulasan makanan Chinese food: 275
Jumlah ulasan makanan Bubur: 161
Jumlah ulasan makanan Masakan Manado: 443
Jumlah ulasan makanan Bakmi: 448
Jumlah ulasan makanan Masakan Nusantara: 3726
Jumlah ulasan makanan France Food: 384
Jumlah ulasan makanan Sundanese Food: 1177
Jumlah ulasan makanan Sop : 260
Jumlah ulasan makanan Steak: 865
Jumlah ulasan makanan Sulawesi Food: 87
Jumlah ulasan makanan Makanan Asia: 39
Jumlah ulasan makanan Snack Nusantara: 35
Jumlah ulasan makanan Makanan korea: 142
Jumlah ulasan makanan Olahan Kambing: 111
Jumlah ulasan makanan Bebek: 305
Jumlah ulasan makanan Gado-gado: 42
Jumlah ulas

### Aspect Sentiment

In [ ]:
aspek_rasa = df['rasa'].unique()
print("Polaritas Rasa:")
print(aspek_rasa)

print(" ")

aspek_penampilan = df['penampilan'].unique()
print("Polaritas Penampilan:")
print(aspek_penampilan)

print(" ")

aspek_sk = df['sensasi_kompleks'].unique()
print("Polaritas Sensasi Kompleks:")
print(aspek_sk)

print(" ")

aspek_tekstur = df['tekstur'].unique()
print("Polaritas Tekstur:")
print(aspek_tekstur)

print(" ")

aspek_aroma = df['aroma'].unique()
print("Polaritas Aroma:")
print(aspek_aroma)

Polaritas Rasa:
['POSITIF' 'NONE' 'NETRAL' 'NEGATIF']
 
Polaritas Penampilan:
['NONE' 'POSITIF' 'NEGATIF' 'NETRAL']
 
Polaritas Sensasi Kompleks:
['NONE' 'POSITIF' 'NETRAL' 'NEGATIF']
 
Polaritas Tekstur:
['NONE' 'POSITIF' 'NEGATIF' 'NETRAL']
 
Polaritas Aroma:
['NONE' 'POSITIF' 'NEGATIF' 'NETRAL']


In [ ]:
x = 0
for polaritas in aspek_rasa:
    print(f"Jumlah review dengan Rasa {aspek_rasa[x]}: {df['rasa'].value_counts().loc[aspek_rasa[x]]}")
    x += 1

Jumlah review dengan Rasa POSITIF: 16457
Jumlah review dengan Rasa NONE: 856
Jumlah review dengan Rasa NETRAL: 653
Jumlah review dengan Rasa NEGATIF: 651


In [ ]:
x = 0
for polaritas in aspek_penampilan:
    print(f"Jumlah review dengan Penampilan {aspek_penampilan[x]}: {df['penampilan'].value_counts().loc[aspek_penampilan[x]]}")
    x += 1

Jumlah review dengan Penampilan NONE: 17246
Jumlah review dengan Penampilan POSITIF: 1031
Jumlah review dengan Penampilan NEGATIF: 333
Jumlah review dengan Penampilan NETRAL: 7


In [ ]:
x = 0
for polaritas in aspek_sk:
    print(f"Jumlah review dengan Sensasi Kompleks {aspek_sk[x]}: {df['sensasi_kompleks'].value_counts().loc[aspek_sk[x]]}")
    x += 1

Jumlah review dengan Sensasi Kompleks NONE: 17671
Jumlah review dengan Sensasi Kompleks POSITIF: 879
Jumlah review dengan Sensasi Kompleks NETRAL: 23
Jumlah review dengan Sensasi Kompleks NEGATIF: 44


In [ ]:
x = 0
for polaritas in aspek_tekstur:
    print(f"Jumlah review dengan Tekstur {aspek_tekstur[x]}: {df['tekstur'].value_counts().loc[aspek_tekstur[x]]}")
    x += 1

Jumlah review dengan Tekstur NONE: 17691
Jumlah review dengan Tekstur POSITIF: 686
Jumlah review dengan Tekstur NEGATIF: 233
Jumlah review dengan Tekstur NETRAL: 7


In [ ]:
x = 0
for polaritas in aspek_aroma:
    print(f"Jumlah review dengan Aroma {aspek_aroma[x]}: {df['aroma'].value_counts().loc[aspek_aroma[x]]}")
    x += 1

Jumlah review dengan Aroma NONE: 18383
Jumlah review dengan Aroma POSITIF: 151
Jumlah review dengan Aroma NEGATIF: 81
Jumlah review dengan Aroma NETRAL: 2


# 2. Pre-processing Data

## 2.1 Case Folding

Mengubah huruf kapital menjadi non-kapital.

In [ ]:
df['ulasan'] = df['ulasan'].str.lower()
df.head()

,no,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma
0,1,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
1,2,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
2,3,sometimes when you feel super hungry especiall...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE
3,4,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
4,5,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE


## 2.2 Cleansing

Menghapus kolom no

In [ ]:
df = df.drop('no', axis=1)

df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
2,sometimes when you feel super hungry especiall...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE
18614,ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE
18615,restoran suasana saung sunda dengan makanan ya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,POSITIF,NONE,NONE


Menghapus Teks Bahasa Inggris yang telah di translate menjadi Bahasa Indonesia

In [ ]:
# Menghapus semua kata sebelum keyword 'kalimat'
keyword = '(diterjemahkan oleh google)'
df['ulasan'] = df['ulasan'].str.replace(rf'.*{keyword}', '', regex=True)

# Menampilkan DataFrame setelah menghapus kata sebelum keyword
df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE
...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE
18614,ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE
18615,restoran suasana saung sunda dengan makanan ya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,POSITIF,NONE,NONE


Menghapus angka, garis baru, spasi, simbol, tanda baca, dan pangkat.

In [ ]:
def remove(text):
    text = re.sub('[0-9]+', '', text) # removing number 0-9
    text = re.sub(r'[\n]+', '', text) # remove new line
    text = re.sub(r"^\s+|\s+$", "", text) # remove leading and trailing spaces in a word using OR sign to delete both
    text = re.sub(r" +", " ", text) # remove multiple space betwen words
    text = re.sub(r'http\S+', '', text) # yang ini yang buat remove https
    text = re.sub('\s+', ' ', text) # removing multiple whitespace
    text = re.sub(r'[^\w\s]', ' ', text) # tidak menghapus word character [a-zA-Z0-9_] dan spasi (emoji dihapus saat ini dijalankan)
    # text = name.replace(/[^a-zA-Z ]/g, "");
    text = text.replace('\\t', ' ').replace('\\n', ' ').replace('\\u', ' ').replace('\\', ' ')#remove tab, new line, and back slice
    text = re.sub(r"²|¹|³|⁰|ⁱ|⁴|⁵|⁶|⁷|⁸|⁹|⁺|⁻|⁽|⁾|ⁿ]+", "", text) # menghapus pangkat
    return text

df['cleaned_ulasan'] = df['ulasan'].apply(lambda x: remove(x))

df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sotonya enak gorengannya juga enak rame teru...
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,strategis n eunak
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE,terkadang ketika anda merasa sangat lapar te...
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enak murah byk cabangny
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sangat baik pelayanannya makanannya juga enak
...,...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,nyaman tempatnya dan luas masakan nya enak har...
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE,kualitas makanam biasa suasana yg lumayan nya...
18614,ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,ya kampung kecil resto lumayan bersih dan meny...
18615,restoran suasana saung sunda dengan makanan ya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,POSITIF,NONE,NONE,restoran suasana saung sunda dengan makanan ya...


Melihat apa ada kata yang memiliki 3 karakter atau lebih yang berulang

In [ ]:
def find_repeated_characters(word):
    pattern = r'(\w)\1{2,}'  # Pola regex untuk mencari tiga atau lebih karakter berulang
    # pattern = r'(\w)\1+'  # Pola regex untuk mencari dua atau lebih karakter berulang
    matches = re.findall(pattern, word)
    return matches

for i, text in enumerate(df['cleaned_ulasan']):
    words = text.split()
    for word in words:
        repeated_characters = find_repeated_characters(word)
        if repeated_characters:
            print(f"Kata '{word}' pada index {i} mengandung karakter berulang: {', '.join(repeated_characters)}")

Kata 'dahhh' pada index 13 mengandung karakter berulang: h
Kata 'sueegerrr' pada index 15 mengandung karakter berulang: r
Kata 'enakksss' pada index 15 mengandung karakter berulang: s
Kata 'enaaak' pada index 18 mengandung karakter berulang: a
Kata 'leckeerrr' pada index 19 mengandung karakter berulang: r
Kata 'sedaaap' pada index 21 mengandung karakter berulang: a
Kata 'josss' pada index 35 mengandung karakter berulang: s
Kata 'enyaaakkkkk' pada index 37 mengandung karakter berulang: a, k
Kata 'bangettt' pada index 64 mengandung karakter berulang: t
Kata 'mihilll' pada index 64 mengandung karakter berulang: l
Kata 'enaaaa' pada index 79 mengandung karakter berulang: a
Kata 'segerrrr' pada index 80 mengandung karakter berulang: r
Kata 'laaah' pada index 80 mengandung karakter berulang: a
Kata 'pelayannnya' pada index 83 mengandung karakter berulang: n
Kata 'blasss' pada index 84 mengandung karakter berulang: s
Kata 'uenakkk' pada index 90 mengandung karakter berulang: k
Kata 'segeeerr'

In [ ]:
i_char3rep = [13, 15, 18, 19, 21, 35]
char3rep_df = df.loc[i_char3rep, :]

char3rep_df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan
13,) ini baik (asli) muanstab dahhh,sebulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,ini baik asli muanstab dahhh
15,sotonya benar2 sueegerrr... enakksss,sebulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,POSITIF,NONE,NONE,sotonya benar sueegerrr enakksss
18,enaaak beut titik,2 bulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enaaak beut titik
19,layanan cepat. murah menu komplit. ada es garu...,2 bulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,layanan cepat murah menu komplit ada es garu...
21,) sangat bagus (asli) sedaaap tenan,2 bulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,POSITIF,sangat bagus asli sedaaap tenan
35,satee dan gorengannya josss,3 bulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,satee dan gorengannya josss


In [ ]:
# menghapus tiga atau lebih pengulangan karakter apa pun, termasuk newline menjadi satu karakter
# contoh: sedaaap -> sedap
def remove_repeated_char3(text):
    pattern = re.compile(r"(\w)\1{2,}")  # Pola regex untuk mencari tiga atau lebih karakter berulang
    return pattern.sub(r"\1", text)

df['del_rep'] = df['cleaned_ulasan'].apply(lambda x: remove_repeated_char3(x))

In [ ]:
i_char3rep = [13, 15, 18, 19, 21, 35]
char3rep_df = df.loc[i_char3rep, :]

char3rep_df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep
13,) ini baik (asli) muanstab dahhh,sebulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,ini baik asli muanstab dahhh,ini baik asli muanstab dah
15,sotonya benar2 sueegerrr... enakksss,sebulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,POSITIF,NONE,NONE,sotonya benar sueegerrr enakksss,sotonya benar sueeger enakks
18,enaaak beut titik,2 bulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enaaak beut titik,enak beut titik
19,layanan cepat. murah menu komplit. ada es garu...,2 bulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,layanan cepat murah menu komplit ada es garu...,layanan cepat murah menu komplit ada es garu...
21,) sangat bagus (asli) sedaaap tenan,2 bulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,POSITIF,sangat bagus asli sedaaap tenan,sangat bagus asli sedap tenan
35,satee dan gorengannya josss,3 bulan lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,satee dan gorengannya josss,satee dan gorengannya jos


## 2.3 Normalization

In [ ]:
# normalization => mengubah kata non-baku menjadi baku menggunakan kbba.txt
# menambahkan kata 'gitoo -> gitu', 'adaa -> ada', 'mikiir -> mikir', 'omong -> bicara', 'by -> oleh', 'embat -> ambil',  
dicts = {row[0] : row[1] for _, row in pd.read_csv('/content/drive/MyDrive/FoodQualityABSA/kbba-edited.txt', delimiter = '\t').iterrows()}
def kbbi(text):
  token = text.split()
  final_string = ' '.join(str(dicts.get(word, word)) for word in token)
  return final_string

df['normalization'] = df['del_rep'].apply(lambda x: kbbi(x))

df.head()

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep,normalization
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak ramai terus...
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,strategis n eunak,strategis n eunak,strategis dan enak
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar teru...
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enak murah byk cabangny,enak murah byk cabangny,enak murah banyak cabangnya
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak


In [ ]:
df.to_csv('checkpoint-1.csv', encoding='utf-8', index=False)

## 2.4 Stopword Removal

menghapus baris yang mengandung noise

In [ ]:
def noise(text):
    text = text.split()
    listStopword = ['lah', 'iya', 'nya', 'sih', 'ya', 'yaa', 'hah', 'wkwk', 'wkwkwk',
                    'wkwkwkwkw', 'wkwkwkwk', 'wuiidih', 'wadidaw', 'he', 'ny', 'eh', 'cm',
                    'kok', 'rb', 'sh'
    ]
    text = [w for w in text if not w in listStopword]
    text = " ".join(text)
    return text

df['stopword'] = df['normalization'].apply(lambda x: noise(x))

df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep,normalization,stopword
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,strategis n eunak,strategis n eunak,strategis dan enak,strategis dan enak
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enak murah byk cabangny,enak murah byk cabangny,enak murah banyak cabangnya,enak murah banyak cabangnya
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan enak hargany...
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...
18614,ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,ya kampung kecil resto lumayan bersih dan meny...,ya kampung kecil resto lumayan bersih dan meny...,iya kampung kecil restoran lumayan bersih dan ...,kampung kecil restoran lumayan bersih dan meny...
18615,restoran suasana saung sunda dengan makanan ya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,POSITIF,NONE,NONE,restoran suasana saung sunda dengan makanan ya...,restoran suasana saung sunda dengan makanan ya...,restoran suasana saung sunda dengan makanan ya...,restoran suasana saung sunda dengan makanan ya...


In [ ]:
def remove_single_char(text):
    return re.sub(r'\b[a-zA-Z]\b', '', text)

df['stopword2'] = df['stopword'].apply(lambda x: noise(x))

df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep,normalization,stopword,stopword2
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,strategis n eunak,strategis n eunak,strategis dan enak,strategis dan enak,strategis dan enak
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enak murah byk cabangny,enak murah byk cabangny,enak murah banyak cabangnya,enak murah banyak cabangnya,enak murah banyak cabangnya
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan enak hargany...,nyaman tempatnya dan luas masakan enak hargany...
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...
18614,ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,ya kampung kecil resto lumayan bersih dan meny...,ya kampung kecil resto lumayan bersih dan meny...,iya kampung kecil restoran lumayan bersih dan ...,kampung kecil restoran lumayan bersih dan meny...,kampung kecil restoran lumayan bersih dan meny...
18615,restoran suasana saung sunda dengan makanan ya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,POSITIF,NONE,NONE,restoran suasana saung sunda dengan makanan ya...,restoran suasana saung sunda dengan makanan ya...,restoran suasana saung sunda dengan makanan ya...,restoran suasana saung sunda dengan makanan ya...,restoran suasana saung sunda dengan makanan ya...


In [ ]:
i_noise = [1224, 1713, 8099, 8394, 11757]
noise_df = df.loc[i_noise, :]

noise_df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep,normalization,stopword,stopword2
1224,enak ngantri wkwkwk,4 bulan lalu,"Waroeng Spesial Sambal ""SS"" Bintaro 1","-6.2731385,106.7461537",44935,Makanan Harian,POSITIF,NONE,NONE,NONE,NONE,enak ngantri wkwkwk,enak ngantri wkwkwk,enak mengantri wkwkwk,enak mengantri,enak mengantri
1713,"harga masih terjangkau, tapi itu roti nya kena...",2 bulan lalu,Blenger Burger,"-6.2411558,106.7900747",44935,Fast Food,NEGATIF,NONE,NONE,NONE,NONE,harga masih terjangkau tapi itu roti nya kena...,harga masih terjangkau tapi itu roti nya kena...,harga masih terjangkau tapi itu roti nya kenap...,harga masih terjangkau tapi itu roti kenapa be...,harga masih terjangkau tapi itu roti kenapa be...
8099,"dekat rumah...rasa enak, harga menyesuaikan wk...",setahun lalu,Pondok Ikan Bakar Ujung Pandang,"-6.2483957,106.7901703",44940,Seafood,POSITIF,NONE,NONE,NONE,NONE,dekat rumah rasa enak harga menyesuaikan wk...,dekat rumah rasa enak harga menyesuaikan wk...,dekat rumah rasa enak harga menyesuaikan wkwkw...,dekat rumah rasa enak harga menyesuaikan semog...,dekat rumah rasa enak harga menyesuaikan semog...
8394,soto padangnya enak sate padangnya enak. dagin...,2 bulan lalu,Soto Padang H. St. Mangkuto,"-6.2470251,106.791828",44940,Soto,POSITIF,NONE,NONE,POSITIF,NONE,soto padangnya enak sate padangnya enak dagin...,soto padangnya enak sate padangnya enak dagin...,soto padangnya enak sate padangnya enak daging...,soto padangnya enak sate padangnya enak daging...,soto padangnya enak sate padangnya enak daging...
11757,wuuuiidddiiihhh.. wadidawwww.. enak mantuls de...,setahun lalu,Double U Steak by Chef Widhi Depok,"-6.3784859,106.8356955",44941,Steak,POSITIF,NONE,NONE,POSITIF,NONE,wuuuiidddiiihhh wadidawwww enak mantuls de...,wuiidih wadidaw enak mantuls deh pokokee ...,wuiidih wadidaw enak mantap betul deh pokokee ...,enak mantap betul deh pokokee saya pesan melti...,enak mantap betul deh pokokee saya pesan melti...


In [ ]:
stopwords.words('indonesian')

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
len(stopwords.words('indonesian'))

758

In [ ]:
# Tokenizing
def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text) 
    return text

# Panggil function tokenizing
df['word_tokenize'] = df['stopword2'].apply(tokenizingText)

In [ ]:
df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep,normalization,stopword,stopword2,word_tokenize
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,"[sotonya, enak, gorengannya, juga, enak, ramai..."
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,strategis n eunak,strategis n eunak,strategis dan enak,strategis dan enak,strategis dan enak,"[strategis, dan, enak]"
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,"[terkadang, ketika, anda, merasa, sangat, lapa..."
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enak murah byk cabangny,enak murah byk cabangny,enak murah banyak cabangnya,enak murah banyak cabangnya,enak murah banyak cabangnya,"[enak, murah, banyak, cabangnya]"
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,"[sangat, baik, pelayanannya, makanannya, juga,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan enak hargany...,nyaman tempatnya dan luas masakan enak hargany...,"[nyaman, tempatnya, dan, luas, masakan, enak, ..."
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,"[kualitas, makanam, biasa, suasana, yang, luma..."
18614,ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,ya kampung kecil resto lumayan bersih dan meny...,ya kampung kecil resto lumayan bersih dan meny...,iya kampung kecil restoran lumayan bersih dan ...,kampung kecil restoran lumayan bersih dan meny...,kampung kecil restoran lumayan bersih dan meny...,"[kampung, kecil, restoran, lumayan, bersih, da..."
18615,restoran suasana saung sunda dengan makanan ya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,POSITIF,NONE,NONE,restoran suasana saung sunda dengan makanan ya...,restoran su

In [ ]:
# Stopword
def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered 
    return text

# Panggil fuction filtering >> stopword
df['stopword3'] = df['word_tokenize'].apply(filteringText)

In [ ]:
df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep,normalization,stopword,stopword2,word_tokenize,stopword3
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,"[sotonya, enak, gorengannya, juga, enak, ramai...","[sotonya, enak, gorengannya, enak, ramai, sian..."
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,strategis n eunak,strategis n eunak,strategis dan enak,strategis dan enak,strategis dan enak,"[strategis, dan, enak]","[strategis, enak]"
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,"[terkadang, ketika, anda, merasa, sangat, lapa...","[terkadang, lapar, jalan, pagi, menemukan, sot..."
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enak murah byk cabangny,enak murah byk cabangny,enak murah banyak cabangnya,enak murah banyak cabangnya,enak murah banyak cabangnya,"[enak, murah, banyak, cabangnya]","[enak, murah, cabangnya]"
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,"[sangat, baik, pelayanannya, makanannya, juga,...","[pelayanannya, makanannya, enak]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan enak hargany...,nyaman tempatnya dan luas masakan enak hargany...,"[nyaman, tempatnya, dan, luas, masakan, enak, ...","[nyaman, tempatnya, luas, masakan, enak, harga..."
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,"[kualitas, makanam, biasa, suasana, yang, luma...","[kualitas, makanam, suasana, lumayan, nyaman, ..."
18614,ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,ya kampung kecil resto lumayan bersih dan meny...,ya kampung kecil resto lumayan bersih dan meny...,iya kampung kecil restoran lumayan bersih dan ...,kampung kecil restoran lumayan bersih dan meny...,kampung kecil restoran lumayan bersih dan meny...,"[k

## 2.6 Stemming

In [ ]:
def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

# gabungin token
df['cleaned_ulasan2'] = df['stopword3'].apply(toSentence)

In [ ]:
df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep,normalization,stopword,stopword2,word_tokenize,stopword3,cleaned_ulasan2
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,"[sotonya, enak, gorengannya, juga, enak, ramai...","[sotonya, enak, gorengannya, enak, ramai, sian...",sotonya enak gorengannya enak ramai siang sian...
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,strategis n eunak,strategis n eunak,strategis dan enak,strategis dan enak,strategis dan enak,"[strategis, dan, enak]","[strategis, enak]",strategis enak
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,"[terkadang, ketika, anda, merasa, sangat, lapa...","[terkadang, lapar, jalan, pagi, menemukan, sot...",terkadang lapar jalan pagi menemukan soto memb...
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enak murah byk cabangny,enak murah byk cabangny,enak murah banyak cabangnya,enak murah banyak cabangnya,enak murah banyak cabangnya,"[enak, murah, banyak, cabangnya]","[enak, murah, cabangnya]",enak murah cabangnya
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,"[sangat, baik, pelayanannya, makanannya, juga,...","[pelayanannya, makanannya, enak]",pelayanannya makanannya enak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan enak hargany...,nyaman tempatnya dan luas masakan enak hargany...,"[nyaman, tempatnya, dan, luas, masakan, enak, ...","[nyaman, tempatnya, luas, masakan, enak, harga...",nyaman tempatnya luas masakan enak harganya st...
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,"[kualitas, makanam, biasa, suasana, yang, luma...","[kualitas, makanam, suasana, lumayan, nyaman, ...",kualitas makanam suasana lumayan nyaman harga ...
18614,ya kampung kecil resto lumayan bersih dan meny...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusant

In [ ]:
# Stemming
def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    # text = [stemmer.stem(word) for word in text]
    text = stemmer.stem(text)
    return text

# Panggil function stemming
df['stemming'] = df['cleaned_ulasan2'].apply(stemmingText)

In [ ]:
df

,ulasan,waktu_ulasan,nama_resto,koordinat_lokasi_resto,tgl_scraping,jenis_makanan,rasa,penampilan,sensasi_kompleks,tekstur,aroma,cleaned_ulasan,del_rep,normalization,stopword,stopword2,word_tokenize,stopword3,cleaned_ulasan2,stemming
0,"sotonya enak, gorengannya juga enak, rame teru...",2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak rame teru...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,sotonya enak gorengannya juga enak ramai terus...,"[sotonya, enak, gorengannya, juga, enak, ramai...","[sotonya, enak, gorengannya, enak, ramai, sian...",sotonya enak gorengannya enak ramai siang sian...,soto enak goreng enak ramai siang siang makan ...
1,strategis n eunak,2 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,strategis n eunak,strategis n eunak,strategis dan enak,strategis dan enak,strategis dan enak,"[strategis, dan, enak]","[strategis, enak]",strategis enak,strategis enak
2,) terkadang ketika anda merasa sangat lapar te...,3 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,NONE,NONE,POSITIF,NONE,NONE,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar te...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,terkadang ketika anda merasa sangat lapar teru...,"[terkadang, ketika, anda, merasa, sangat, lapa...","[terkadang, lapar, jalan, pagi, menemukan, sot...",terkadang lapar jalan pagi menemukan soto memb...,terkadang lapar jalan pagi temu soto bunuh lap...
3,enak murah byk cabangny,5 hari lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,enak murah byk cabangny,enak murah byk cabangny,enak murah banyak cabangnya,enak murah banyak cabangnya,enak murah banyak cabangnya,"[enak, murah, banyak, cabangnya]","[enak, murah, cabangnya]",enak murah cabangnya,enak murah cabang
4,"sangat baik pelayanannya, makanannya juga enak",seminggu lalu,Soto Sedaap Boyolali Hj. Hesti Widodo Cab Tangsel,"-6.3154391,106.7036096",44935,Soto,POSITIF,NONE,NONE,NONE,NONE,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,sangat baik pelayanannya makanannya juga enak,"[sangat, baik, pelayanannya, makanannya, juga,...","[pelayanannya, makanannya, enak]",pelayanannya makanannya enak,layan makan enak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18612,nyaman tempatnya dan luas masakan nya enak har...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,POSITIF,NONE,NONE,NONE,NONE,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan nya enak har...,nyaman tempatnya dan luas masakan enak hargany...,nyaman tempatnya dan luas masakan enak hargany...,"[nyaman, tempatnya, dan, luas, masakan, enak, ...","[nyaman, tempatnya, luas, masakan, enak, harga...",nyaman tempatnya luas masakan enak harganya st...,nyaman tempat luas masakan enak harga standart
18613,kualitas makanam biasa. suasana yg lumayan nya...,setahun lalu,Kampung Kecil Restoran Cab. Bojongsari Depok,"-6.3926952,106.7435701",44942,Masakan Nusantara,NETRAL,NONE,NONE,NONE,NONE,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yg lumayan nya...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,kualitas makanam biasa suasana yang lumayan ny...,"[kualitas, makanam, biasa, suasana, yang, luma...","[kualitas, makanam, suasana, lumayan, nyaman, ..."

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18617 entries, 0 to 18616
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ulasan                  18617 non-null  object
 1   waktu_ulasan            18617 non-null  object
 2   nama_resto              18617 non-null  object
 3   koordinat_lokasi_resto  18617 non-null  object
 4   tgl_scraping            18617 non-null  int64 
 5   jenis_makanan           18617 non-null  object
 6   rasa                    18617 non-null  object
 7   penampilan              18617 non-null  object
 8   sensasi_kompleks        18617 non-null  object
 9   tekstur                 18617 non-null  object
 10  aroma                   18617 non-null  object
 11  cleaned_ulasan          18617 non-null  object
 12  del_rep                 18617 non-null  object
 13  normalization           18617 non-null  object
 14  stopword                18617 non-null  object
 15  st

In [ ]:
df.to_csv('checkpoint-2-stemming.csv', encoding='utf-8', index=False)

# SVM TF-IDF

In [ ]:
# Membuat vektor fitur menggunakan TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
# Bagi data menjadi data latih dan data uji
X = df['ulasan']  # Kolom ulasan sebagai fitur
y = df[['rasa', 'penampilan', 'sensasi_kompleks', 'tekstur', 'aroma']]  # Kolom aspek dan polaritas sebagai target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Membuat dan melatih model SVM untuk setiap aspek
models = {}
for aspect in y.columns:
    model = SVC(kernel='linear')
    model.fit(X_train_vectorized, y_train[aspect])
    models[aspect] = model

In [ ]:
model

SVC(kernel='linear')

In [ ]:
# Melakukan prediksi pada data uji
y_pred = {}
for aspect, model in models.items():
    y_pred[aspect] = model.predict(X_test_vectorized)

In [ ]:
# Menampilkan laporan hasil klasifikasi
for aspect in y.columns:
    print(f"Laporan hasil klasifikasi untuk aspek {aspect}:")
    print(classification_report(y_test[aspect], y_pred[aspect]))
    print()

Laporan hasil klasifikasi untuk aspek rasa:
              precision    recall  f1-score   support

     NEGATIF       0.80      0.40      0.54       129
      NETRAL       0.85      0.56      0.67       126
        NONE       0.81      0.32      0.46       172
     POSITIF       0.93      0.99      0.96      3297

    accuracy                           0.93      3724
   macro avg       0.85      0.57      0.66      3724
weighted avg       0.92      0.93      0.91      3724


Laporan hasil klasifikasi untuk aspek penampilan:
              precision    recall  f1-score   support

     NEGATIF       0.90      0.39      0.54        67
      NETRAL       0.00      0.00      0.00         2
        NONE       0.96      1.00      0.98      3440
     POSITIF       0.81      0.41      0.54       215

    accuracy                           0.95      3724
   macro avg       0.67      0.45      0.52      3724
weighted avg       0.95      0.95      0.94      3724


Laporan hasil klasifikasi untuk as

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

              precision    recall  f1-score   support

     NEGATIF       0.00      0.00      0.00         7
      NETRAL       0.00      0.00      0.00         4
        NONE       0.96      1.00      0.98      3542
     POSITIF       0.78      0.19      0.30       171

    accuracy                           0.96      3724
   macro avg       0.44      0.30      0.32      3724
weighted avg       0.95      0.96      0.94      3724


Laporan hasil klasifikasi untuk aspek tekstur:
              precision    recall  f1-score   support

     NEGATIF       0.92      0.39      0.54        57
      NETRAL       0.00      0.00      0.00         2
        NONE       0.98      1.00      0.99      3518
     POSITIF       0.92      0.71      0.80       147

    accuracy                           0.98      3724
   macro avg       0.70      0.52      0.58      3724
weighted avg       0.98      0.98      0.97      3724


Laporan hasil klasifikasi untuk aspek aroma:
              precision    recall  f

### error

In [ ]:
# # Encode target data into numerical values
# polarity_encode = {'NEGATIF' : 0, 'POSITIF' : 1, 'NETRAL' : 2, 'NONE' : 3}
# y = df[['rasa', 'penampilan', 'sensasi_kompleks', 'tekstur', 'aroma']].replace(polarity_encode).values

In [ ]:
# y

array([[1, 3, 3, 3, 3],
       [1, 3, 3, 3, 3],
       [3, 3, 1, 3, 3],
       ...,
       [1, 3, 3, 3, 3],
       [1, 3, 1, 3, 3],
       [1, 3, 3, 3, 3]])

In [ ]:
# # Bagi data menjadi data latih dan data uji
# X = df['ulasan']  # Kolom ulasan sebagai fitur
# # y = df[['rasa', 'penampilan', 'sensasi_kompleks', 'tekstur', 'aroma']]  # Kolom aspek dan polaritas sebagai target
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# X

0        sotonya enak, gorengannya juga enak, rame teru...
1                                        strategis n eunak
2        ) terkadang ketika anda merasa sangat lapar te...
3                                  enak murah byk cabangny
4           sangat baik pelayanannya, makanannya juga enak
                               ...                        
18612    nyaman tempatnya dan luas masakan nya enak har...
18613    kualitas makanam biasa. suasana yg lumayan nya...
18614    ya kampung kecil resto lumayan bersih dan meny...
18615    restoran suasana saung sunda dengan makanan ya...
18616                     makanannya enak tempatnya nyaman
Name: ulasan, Length: 18617, dtype: object

In [ ]:
# X_train

4894     seperti biasa normatif harga n rasa makanannya...
12430                                                 enak
7728     ) makanan indonesia dimoderasi untuk selera or...
280      makanannya enak, tempatnya bersih, variasinya ...
10228    rasa n harga emang bintang 5, tp pelayanan nya...
                               ...                        
11284                               pecak gurame nya juara
11964     ) area bersih dan makanan enak ... hmmm enak ...
5390     ) sajian warisan jakarta sinve 1970 menurut sa...
860      pelayanannya bagus dan ramah sekali. makananny...
15795          tempat nyaman, bebek nyamie, sambel nyoz...
Name: ulasan, Length: 14893, dtype: object

In [ ]:
# X_test

13152                             enak,sesuai hargalahh...
409      ayam 1 potong di ceburin ke kuah santan dengan...
13142                                enak lah pokoknya 😂👍🏻
8922                                   mahal dan biasa aja
5847     salah satu mie langganan. mie nya aga tebal, b...
                               ...                        
14778    enak yg jelas ga menguras kantong, tapi rasany...
6165     sesuai ekspektasi....cocok utk lidah jawa,kare...
13442                                       enak dan murah
9542                                              mantappp
13863    daging satenya besar-besar, dapat dinikmati de...
Name: ulasan, Length: 3724, dtype: object

In [ ]:
# y_train

array([[1, 3, 3, 3, 3],
       [1, 3, 3, 3, 3],
       [1, 3, 3, 3, 3],
       ...,
       [1, 3, 3, 3, 3],
       [1, 3, 1, 3, 3],
       [1, 3, 3, 3, 3]])

In [ ]:
# y_test

array([[1, 3, 3, 3, 3],
       [1, 3, 3, 3, 3],
       [1, 3, 3, 3, 3],
       ...,
       [1, 3, 3, 3, 3],
       [1, 3, 3, 3, 3],
       [3, 1, 3, 3, 3]])

In [ ]:
# # Ekstraksi fitur menggunakan TF-IDF
# vectorizer = TfidfVectorizer()
# X_train_vectorized = vectorizer.fit_transform(X_train)
# X_test_vectorized = vectorizer.transform(X_test)

In [ ]:
# X_train_vectorized

<14893x14537 sparse matrix of type '<class 'numpy.float64'>'
	with 212377 stored elements in Compressed Sparse Row format>

In [ ]:
# X_test_vectorized

<3724x14537 sparse matrix of type '<class 'numpy.float64'>'
	with 51998 stored elements in Compressed Sparse Row format>

In [ ]:
# # Membuat dan melatih model SVM untuk setiap aspek
# models = {}
# for aspect in y.columns:
#     model = SVC(kernel='linear')
#     model.fit(X_train_vectorized, y_train[aspect])
#     models[aspect] = model

AttributeError: ignored

In [ ]:
# # Melakukan prediksi pada data uji untuk setiap aspek
# y_pred = {}
# for aspect, model in models.items():
#     y_pred[aspect] = model.predict(X_test_vectorized)



In [ ]:
# # Evaluasi model untuk setiap aspek
# for aspect in y.columns:
#     print(f"=== Aspek: {aspect} ===")
#     print(classification_report(y_test[aspect], y_pred[aspect]))
#     print()

# Word Embedding

## Contextual BERT Embeddings

In [ ]:
# # Load pre-trained model tokenizer (vocabulary)
# tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")

In [ ]:
text = "Here is the sentence I want embeddings for."
marked_text = "[CLS] " + text + " [SEP]"

# Tokenize our sentence with the BERT tokenizer.
tokenized_text = tokenizer.tokenize(marked_text)

# Print out the tokens.
print (tokenized_text)

In [ ]:
X = df['stemming']  # Kolom ulasan sebagai fitur
y = df[['rasa', 'penampilan', 'sensasi_kompleks', 'tekstur', 'aroma']]  # Kolom aspek dan polaritas sebagai target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("indolem/indobert-base-uncased", num_labels=len(y.columns))


In [ ]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(y_train.values, dtype=torch.float))

test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             torch.tensor(y_test.values, dtype=torch.float))

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)



---

